In [1]:
import pandas as pd
import numpy as np

import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns

import matplotlib.pyplot as plt

# to play the audio files
from IPython.display import Audio
from statistics import mean, stdev
import imblearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
Ravdess = "data/audio_speech_actors_01-24/"
Savee = "savee/ALL/"
emodb = "emodb/wav/"
Tess = "../archive/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/"

In [3]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(Tess + dir)
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(Tess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
df = pd.concat([emotion_df, path_df], axis=1)
df.head()

,Emotions,Path
0,angry,../archive/TESS Toronto emotional speech set d...
1,angry,../archive/TESS Toronto emotional speech set d...
2,angry,../archive/TESS Toronto emotional speech set d...
3,angry,../archive/TESS Toronto emotional speech set d...
4,angry,../archive/TESS Toronto emotional speech set d...


In [4]:
Y = []
for i in df.Emotions[:]:
    Y.append(i)

In [5]:
len(Y)

2800

In [6]:
paths = []

for i in range(2800):
    paths.append(df['Path'][i])

In [ ]:
paths

In [8]:
from tqdm import tqdm

In [9]:
import opensmile

X = pd.DataFrame()

# Define the feature set you want to use
feature_set = opensmile.FeatureSet.ComParE_2016

# Initialize opensmile with the chosen feature set
smile = opensmile.Smile(
    feature_set=feature_set,
    feature_level=opensmile.FeatureLevel.Functionals,
)

for i in tqdm(paths):
    features = smile.process_file(i)
    # features_transposed = features.transpose()  # Transpose the DataFrame
    X = pd.concat([X, features], axis=0, ignore_index=True)

# # Reset column names to integer index
X.columns = range(X.shape[1])

100%|██████████| 2800/2800 [05:28<00:00,  8.53it/s]


In [10]:
X

,0,1,2,3,4,5,6,7,8,9,...,6363,6364,6365,6366,6367,6368,6369,6370,6371,6372
0,1.409512,0.452055,0.000000,0.204863,0.589691,1.011887,0.384828,0.422196,0.807024,0.030353,...,5.348729,0.289192,4.242554,4.327974,-17.914936,0.608820,152.885727,83.054291,172.459793,75.942635
1,0.916712,0.188312,0.701299,0.184017,0.375766,0.684931,0.191749,0.309165,0.500914,0.016434,...,6.557486,0.334924,3.963181,4.109989,-15.706089,0.554739,190.921707,93.375954,165.881805,115.629372
2,1.463523,0.170068,0.693878,0.174246,0.380847,0.798810,0.206601,0.417963,0.624564,0.017857,...,9.870838,0.476234,4.304881,4.323459,-19.924049,0.498563,180.678787,100.956093,193.420212,131.130020
3,1.530983,0.169935,0.692810,0.178458,0.553597,0.974556,0.375139,0.420959,0.796099,0.015183,...,4.892866,0.271253,3.832680,3.775176,18.664227,0.740595,140.893097,87.829964,146.642090,73.038651
4,1.358880,0.174242,0.696970,0.267951,0.484316,0.670793,0.216365,0.186477,0.402842,0.024349,...,7.889145,0.459729,3.824991,3.944466,-16.407700,0.500121,199.191910,101.266716,98.194481,41.534412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,1.185210,0.200000,0.646512,0.204553,0.371865,0.602021,0.167312,0.230156,0.397468,0.074873,...,5.458078,0.457947,2.452641,2.440002,19.871544,0.504870,123.588638,59.071106,97.787613,58.579617
2796,1.072646,0.179612,0.718447,0.216095,0.404453,0.597532,0.188358,0.193079,0.381437,0.042804,...,6.242836,0.525422,2.299856,2.361291,-16.995899,0.455454,109.777870,67.006943,88.109619,51.920315
2797,0.980292,0.209302,0.693023,0.215134,0.384987,0.728419,0.169853,0.343431,0.513285,0.085830,...,6.561334,0.509982,2.274836,2.306772,-18.804390,0.464993,111.616272,62.557739,86.448982,50.563286
2798,1.047530,0.181416,0.654867,0.176108,0.254267,0.579653,0.078159,0.325386,0.403545,0.046640,...,4.526168,0.428460,2.143402,2.171680,-18.926029,0.596808,100.704506,52.543175,87.989960,41.919945


In [11]:
Features = X
Features['labels'] = Y

In [12]:
Features.head()
Features.to_csv('../feature_csvs/tess_features.csv', index=False)

In [15]:
Features = pd.read_csv('tess_features.csv')
Features.head()
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

In [16]:
X.shape, Y.shape

((2800, 6373), (2800,))

In [17]:
Y.size

2800

In [18]:
type(Y)

numpy.ndarray

In [19]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [20]:
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.1,random_state=4, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Step 1: Convert one-hot encoded labels back to numerical labels
y_train = y_train.argmax(axis=1)
y_test = y_test.argmax(axis=1)

In [21]:
Y.shape

(2800, 7)

In [22]:
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score
# Step 2: Create an SVM classifier
svm_classifier = svm.SVC(kernel='rbf', C=100)  # You can choose different parameters based on your data

# Step 3: Train the SVM classifier
svm_classifier.fit(x_train, y_train)

# Step 4: Make predictions on the test set
y_test_pred = svm_classifier.predict(x_test)

# Step 5: Evaluate the model on the test set
precision = accuracy_score(y_test, y_test_pred)
print("Test f1:", precision)

Test f1: 0.9964285714285714


In [24]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=5, random_state=4)

# Train the classifier on the training data
rf_classifier.fit(x_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(x_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.98


In [28]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(n_estimators=10, random_state=4)

# Train the classifier on the training data
xgb_classifier.fit(x_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(x_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.72


In [25]:
emotion_dict = {
    0: 'neutral',
    1: 'calm',
    2: 'happy',
    3: 'sad',
    4: 'angry',
    5: 'fearful',
    6: 'disgust',
    7: 'surprised'
}

In [26]:
from sklearn.metrics import classification_report
y_true_emotions = np.array([emotion_dict[label] for label in y_test])
y_pred_emotions = np.array([emotion_dict[label] for label in y_test_pred])

# Generate a new classification report
report_with_emotions = classification_report(y_true_emotions, y_pred_emotions)

# Display the new classification report
print(report_with_emotions)

              precision    recall  f1-score   support

       angry       1.00      1.00      1.00        46
        calm       1.00      1.00      1.00        48
     disgust       0.98      1.00      0.99        43
     fearful       1.00      1.00      1.00        29
       happy       1.00      1.00      1.00        38
     neutral       1.00      1.00      1.00        37
         sad       1.00      0.97      0.99        39

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280



In [27]:
x_train_df = pd.DataFrame(x_train)
x_test_df = pd.DataFrame(x_test)
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)

In [28]:
def objective_knn(features):
    features = features.astype(int)
    X_train_subset = x_train_df.iloc[:, features]
    X_test_subset = x_test_df.iloc[:, features]
    X_test_subset = np.array(X_test_subset)  # Ensure X_test_subset is a NumPy array
    if not X_test_subset.flags.c_contiguous:  # Check contiguity and make it contiguous if necessary
        X_test_subset = np.ascontiguousarray(X_test_subset)
    knn_classifier = KNeighborsClassifier(n_neighbors=1)
    knn_classifier.fit(X_train_subset, y_train)
    y_pred = knn_classifier.predict(X_test_subset)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [29]:
objective_knn(np.array(list(range(6372))))

0.9642857142857143

In [30]:
from sklearn.feature_selection import mutual_info_classif

In [31]:
# objective_info_gain(np.array(list(range(100))))

In [ ]:
from mealpy import IntegerVar,GA,DE,CRO,EP,HS,WCA,AEO,GCO,FA,QSA,SHADE

def objective_knn(features):
    features = features.astype(int)
    X_train_subset = x_train_df.iloc[:, features]
    X_test_subset = x_test_df.iloc[:, features]
    X_test_subset = np.array(X_test_subset)  # Ensure X_test_subset is a NumPy array
    if not X_test_subset.flags.c_contiguous:  # Check contiguity and make it contiguous if necessary
        X_test_subset = np.ascontiguousarray(X_test_subset)
    knn_classifier = KNeighborsClassifier(n_neighbors=1)
    knn_classifier.fit(X_train_subset, y_train)
    y_pred = knn_classifier.predict(X_test_subset)
    acc = accuracy_score(y_test, y_pred)
    return acc

def objective_info_gain(features):
    features = features.astype(int)
    X_train_subset = x_train_df.iloc[:, features]
    info_gain = mutual_info_classif(X_train_subset, y_train)  # Calculate information gain for each feature
    return np.sum(info_gain)  # Sum of information gains of selected features

def objective_svm(features):
    features = features.astype(int)
    X_train_subset = x_train_df.iloc[:, features]
    X_test_subset = x_test_df.iloc[:, features]
    svm_classifier_rbf = svm.SVC(kernel='rbf', C=100)
    svm_classifier_rbf.fit(X_train_subset, y_train)
    y_pred = svm_classifier_rbf.predict(X_test_subset)
    acc = accuracy_score(y_test, y_pred)
    return acc

problem_dict = {
    "obj_func": objective_svm,
    "bounds": IntegerVar(lb=[0, ] * 50, ub=[6372, ] * 50,),
    "minmax": "max",
}

# optimizer = GA.BaseGA(epoch=100, pop_size=64, pc=0.95, pm=0.1)
# optimizer = CRO.OCRO(epoch=100, pop_size=64, po = 0.4, Fb = 0.9, Fa = 0.1, Fd = 0.1, Pd = 0.5, GCR = 0.1, gamma_min = 0.02, gamma_max = 0.2, n_trials = 5, restart_count = 50)
# optimizer = DE.JADE(epoch=1000, pop_size=50, miu_f = 0.5, miu_cr = 0.5, pt = 0.1, ap = 0.1)  # decent
# optimizer = HS.DevHS(epoch=100, pop_size=64, c_r = 0.95, pa_r = 0.05)
# optimizer = AEO.AugmentedAEO(epoch=100, pop_size=64)    good
# optimizer =  WCA.OriginalWCA(epoch=1000, pop_size=64, nsr = 4, wc = 2.0, dmax = 1e-6) # good WCA
# optimizer = FA.OriginalFA(epoch=100, pop_size=64, max_sparks = 50, p_a = 0.04, p_b = 0.8, max_ea = 40, m_sparks = 50) #   decent
# optimizer =   QSA.ImprovedQSA(epoch=1000, pop_size=64)  alright
# optimizer = SHADE.L_SHADE(epoch=1000, pop_size=50, miu_f = 0.5, miu_cr = 0.5)

optimizer = GA.BaseGA(epoch=100, pop_size=64, pc=0.95, pm=0.1)
optimizer.solve(problem_dict)

print(optimizer.g_best.solution)
print(optimizer.g_best.target.fitness)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def objective_rf(features):
    features = features.astype(int)
    X_train_subset = x_train_df.iloc[:, features]
    X_test_subset = x_test_df.iloc[:, features]
    random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    random_forest_classifier.fit(X_train_subset, y_train)
    y_pred = random_forest_classifier.predict(X_test_subset)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [34]:
main_sol = optimizer.g_best.solution
main_sol_int = main_sol.astype(int)
np.save('tess_50_opensm_svm_100percent.npy',main_sol_int)

In [35]:
# a = np.load('opensmile_100_svm.npy')
objective_rf(main_sol_int)

0.9857142857142858

In [36]:
def objective_svm_f1(features):
    features = features.astype(int)
    X_train_subset = x_train_df.iloc[:, features]
    X_test_subset = x_test_df.iloc[:, features]
    svm_classifier_rbf = svm.SVC(kernel='rbf', C=100)
    svm_classifier_rbf.fit(X_train_subset, y_train)
    y_pred = svm_classifier_rbf.predict(X_test_subset)
    from sklearn.metrics import classification_report
    y_true_emotions = np.array([emotion_dict[label] for label in y_test])
    y_pred_emotions = np.array([emotion_dict[label] for label in y_pred])

    # Generate a new classification report
    report_with_emotions = classification_report(y_true_emotions, y_pred_emotions)

    # Display the new classification report
    print(report_with_emotions)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return f1

In [78]:
a = np.array([4484, 5220, 5819, 2352, 3971, 2798, 2313, 896, 3702, 142, 1838, 1827, 2192, 3207, 282, 1754, 4816, 5308, 6300, 4397, 3662, 1815, 4390, 1893, 2778, 37, 3713, 6366, 6255, 2336, 1322, 3692, 3673, 5667, 1191, 3962, 1490, 4634, 1139, 96, 5045, 4854, 2934, 1530, 801, 2282, 1858, 1671, 5172, 5424, 4752, 1849, 2073, 4591, 4721, 5811, 3250, 448, 4790, 784, 6230, 372, 3551, 4452, 3902, 4560, 1543, 4201, 5737, 1114, 4208, 4561, 2847, 2267, 3778, 4032, 3714, 5418, 1426, 4756, 4190, 5746, 4416, 3567, 2319, 4644, 3017, 5219, 3525, 2573, 671, 1218, 3423, 1068, 3954, 464, 395, 5613, 1664, 2600, 1015, 5405, 5177, 1791, 5265, 459, 5369, 2877, 3677, 4141, 4865, 5669, 4159, 2608, 3423, 1626, 3239, 6198, 4548, 2877, 4313, 4861, 5005, 3314, 4271, 2241, 6174, 461, 6249, 1453, 2076, 2081, 2661, 2005, 5283, 3867, 5878, 4057, 2380, 4242, 6261, 2336, 2950, 2240, 6199, 1747, 1378, 157, 4612, 3211, 4616, 5619, 5258, 566, 5309, 5998, 5652, 5180, 5304, 1649, 2695, 3058, 374, 5006, 5273, 1530, 5141, 4364, 3166, 3062, 4816, 3516, 3731, 4304, 2648, 2392, 5906, 4761, 4795, 865, 2638, 4325, 839, 9, 989, 5697, 786, 2907, 1170, 4375, 6344, 1075, 349, 1688, 4483, 1864, 3855, 5546, 1441, 2551, 2425, 5540, 796, 1128, 5447, 5381, 4837, 2253, 3176, 4984, 5121, 241, 3731, 5881, 973, 5668, 2135, 4301, 1927, 2632, 5561, 1090, 1609, 5783, 2357, 2564, 606, 3756, 3912, 920, 2128, 218, 5022, 1663, 1038, 1714, 1046, 2908, 5587, 5819, 5546, 2236, 3756, 766, 1271, 5582, 3054, 3885, 3142, 609])

In [79]:
b = np.array([5353, 4170, 510, 5395, 1754, 1581, 5875, 159, 5213, 2843, 3337, 2979, 2465, 5740, 6175, 125, 2175, 4568, 2170, 199, 1455, 5297, 4354, 3537, 837, 4960, 3664, 945, 4908, 3396, 1771, 3278, 2368, 5418, 6220, 3881, 2369, 1882, 5537, 2441, 1559, 4238, 5684, 5579, 4025, 4812, 433, 6225, 2076, 3490, 5034, 2273, 1838, 4010, 4236, 1188, 2398, 601, 3666, 1786, 1400, 5757, 1526, 4559, 242, 2448, 6204, 1149, 4830, 5632, 297, 4705, 2358, 994, 5792, 2316, 966, 1957, 824, 15, 5116, 3689, 696, 3238, 2406, 1678, 6199, 192, 3666, 3694, 1943, 1476, 6012, 348, 1801, 6155, 1843, 3679, 5359, 2068, 2361, 4569, 220, 3857, 2322, 3528, 6020, 1762, 3605, 5822, 4790, 1820, 975, 4714, 5870, 3292, 2966, 1094, 6100, 6210, 2846, 5343, 1647, 2127, 5030, 3026, 6180, 6329, 1969, 5529, 1524, 1396, 530, 2116, 1989, 2607, 1424, 4869, 1573, 5189, 5242, 5308, 1559, 6174, 815, 6161, 5136, 3306, 1729, 1167, 5817, 5914, 3283, 1878, 2208, 3595, 207, 2422, 1087, 3233, 1557, 5078, 1654, 651, 5767, 6289, 1681, 6247, 77, 3917, 2701, 2980, 3679, 6013, 1401, 5267, 3678, 228, 2666, 5557, 49, 44, 4963, 1870, 3635, 3458, 3923, 1961, 521, 5865, 3862, 4494, 3855, 4899, 1341, 1377, 1942, 982, 2792, 1739, 5286, 5986, 5119, 1216, 842, 1653, 5540, 239, 3932, 2924, 6013, 1856, 1720, 3165, 3401, 5237, 3515, 5507, 2552, 5356, 3395, 618, 3431, 2939, 517, 5903, 3022, 994, 1144, 877, 2007, 2390, 1610, 3146, 1268, 5501, 257, 854, 2572, 3210, 294, 804, 2092, 4828, 1041, 1499, 70, 3305, 642, 792])

In [37]:
objective_svm_f1(main_sol_int)

              precision    recall  f1-score   support

       angry       1.00      1.00      1.00        46
        calm       1.00      1.00      1.00        48
     disgust       1.00      1.00      1.00        43
     fearful       1.00      1.00      1.00        29
       happy       1.00      1.00      1.00        38
     neutral       1.00      1.00      1.00        37
         sad       1.00      1.00      1.00        39

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280



1.0

In [38]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
import datetime
from statistics import mean, stdev
import imblearn

In [39]:
Y = Features['labels'].values
data_1 = X[:,main_sol_int]

# data_1.shape
labels=Y

oversample = imblearn.over_sampling.SMOTE()
data, labels = oversample.fit_resample(data_1, Y)

sc = StandardScaler()
x_scaled = sc.fit_transform(data)

y = np.asarray(labels)

In [ ]:
svclassifier = svm.SVC(kernel='rbf', C=100)

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=4)  # 1 -- 77.91, 4 -- 78
lst_accu_stratified = []
times = []
y_actual_list = []
y_pred_list = []

for train_index, test_index in skf.split(x_scaled, labels):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    a = datetime.datetime.now()
    svclassifier.fit(x_train_fold, y_train_fold)
    b = datetime.datetime.now()
    lst_accu_stratified.append(svclassifier.score(x_test_fold, y_test_fold))

    y_pred = svclassifier.predict(x_test_fold)

    times.append((b-a))
    #add actual and pred labels to lists
    y_actual_list.append(y_test_fold.copy())
    y_pred_list.append(y_pred.copy())

# s = np.zeros((8,8))

for i in range(len(y_actual_list)):
    print("Fold ",i,"..................")
    print("accuracy = ", lst_accu_stratified[i])
    #print(classification_report(y_actual_list[i], y_pred_list[i]))
    #print(confusion_matrix(y_actual_list[i], y_pred_list[i]))
#     s += confusion_matrix(y_actual_list[i], y_pred_list[i])

#print(s)
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified) * 100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified) * 100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified) * 100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))
print("average time taken by a fold = ", np.asarray(times).mean())

In [43]:
y

array(['angry', 'angry', 'angry', ..., 'sad', 'sad', 'sad'], dtype=object)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize

# # Load iris dataset
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target

# Binarize the labels for multiclass classification
y_bin = label_binarize(y, classes=np.unique(y))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=4, shuffle=True)

# Train SVM classifier
clf = SVC(kernel='rbf', C= 100,decision_function_shape='ovr')
clf.fit(X_train, y_train)

# Get decision scores for each sample
decision_scores = clf.decision_function(X_test)

# Plot box plot
plt.figure(figsize=(10, 6))
plt.boxplot(decision_scores, patch_artist=True, showmeans=True)
plt.xticks(np.arange(1, len(np.unique(y))+1), list(emotion_dict))
plt.xlabel('Class')
plt.ylabel('Decision Score')
plt.title('Box Plot of Decision Scores for SVM Multiclass Classification')
plt.grid(True)
plt.show()
